# Testing Notebook
This notebook will be used for carrying out test cases as well as explaining/logicing the steps of the 2 phase simplex method employed in this repo. \
An understanding of the phase 1 simplex method is required to understand the following (as explaining/deriving everything would take a very long time).

## Tableau Construction
Here we will justify and test how the tableau construction will work in our program.

### Tableau Construction for the 2 phase simplex method

Here our problems are solveable from the following form:\
$  \text{min }c\cdot x$ \
$  \text{s.t. } Ax = b$ \
$  x \geq 0$ 
\
\
Our auxllary problem (to solve problems with no apparent basis, i.e. no apparent identity basic matrix fromed in matrix A) is then: \
$  \text{min } e\cdot y$ \
$  \text{s.t. } Ax + I_n y = b$ \
$  x \geq 0, y \geq 0$ \
$ e = (1,...,1)^T$
\
\
Hence our problem for the auxillary problem can be proven to be the following: \
$\text{min } e\cdot(b-Ax)$ \
$  \text{s.t. } Ax + I_n y = b$ \
$  x \geq 0, y \geq 0$ \
$ e = (1,...,1)^T$ \
\
So our tableau becomes:
| $-e\cdot b$| $-eA$ | $0...0$ |
| :----------:| :-----:| :----: |
| $b$        |   $A$  | $I$ |

Notice the following facts:
1. Our tableau can be represented as a matrix of size $(m+1) \times (m+n+1)$
2. This tableau can be constructed numerically

We use numpy.linalg to compute matrix multiplication as prescribed above. \
Note that we only have a valid solution if the y variables are all 0, otherwise the original problem's constraints are violated.

### Tableau construction tests

In [11]:
from two_phase_simplex import TwoPhaseSimplex

# test the basic tableau construction of the auxillary problem
# simple 2x2 matrix case
A = [[1,0],[1,1]]
b = [2,3]
c = [1,1]
x = TwoPhaseSimplex(A,b,c)
x.solve_program()
print(x.get_tableau())

# non-square matrix
A = [[3,2,2],[4,5,4],[2,1,1],[0,1,1]]
b = [1,2,3,4]
c = [-1,-2,3]
y = TwoPhaseSimplex(A,b,c)
y.solve_program()
print(y.get_tableau())

# check if these two outputs check out

[array([0., 0., 0., 1., 1.]), array([2., 1., 0., 1., 0.]), array([ 1.,  0.,  1., -1.,  1.])]
[array([-6.,  1.,  0.,  0.,  2.,  1.,  0.,  0.]), array([ 0.5,  3.5,  0. ,  1. ,  2.5, -1. ,  0. ,  0. ]), array([ 0., -2.,  1.,  0., -2.,  1.,  0.,  0.]), array([ 2.5,  0.5,  0. ,  0. , -0.5,  0. ,  1. ,  0. ]), array([ 3.50000000e+00, -1.50000000e+00,  0.00000000e+00,  0.00000000e+00,
       -5.00000000e-01,  5.55111512e-17,  0.00000000e+00,  1.00000000e+00])]


### Tableau feasibility tests

#### Feasiblity via violation of auxillary problems non-negativity constaint